# <span id="0">Casus *Hidden Markov Model* - Deel II</span>

Inhoud:

* **<a href="#1">Menti</a>**

* **<a href="#2">Kansrekening</a>**

* **<a href="#3">Experimentele waarschijnlijkheid</a>**

* **<a href="#4">Je eigen `HiddenMarkovModel` class</a>**

* **<a href="#5">CpG-eilandjes</a>**

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
import numpy as np

<a id="1" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### Menti

De onderstaande Menti gaat over kansrekening. We voeren deze uit tijdens de les.

In [2]:
%%html
<div style='position: relative; padding-bottom: 56.25%; padding-top: 35px; height: 0; overflow: hidden;'><iframe sandbox='allow-scripts allow-same-origin allow-presentation' allowfullscreen='true' allowtransparency='true' frameborder='0' height='315' src='https://www.mentimeter.com/app/presentation/alao6y5iwixjzjyycjfe8hcfqk7jy199/embed' style='position: absolute; top: 0; left: 0; width: 100%; height: 100%;' width='420'></iframe></div>

In [3]:
# UITWERKING

<a id="2" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### Kansrekening

Kansrekening is een tak van de wiskunde die zich richt op het begrijpen en kwantificeren van onzekerheid en willekeurige verschijnselen. Stel je voor dat je een dobbelsteen gooit: de uitkomst is onzeker en kan elk van de zes zijden laten zien. Kansrekening helpt ons om de waarschijnlijkheid van deze mogelijke uitkomsten te beschrijven.

Een basisconcept in kansrekening is dat van een experiment en de mogelijke uitkomsten ervan. In ons voorbeeld is het gooien van een dobbelsteen het experiment, en de mogelijke uitkomsten zijn de getallen 1 tot en met 6. Een gebeurtenis is een specifieke set van uitkomsten die ons interesseert, zoals het gooien van een even getal.

De kans van een gebeurtenis is een getal tussen 0 en 1 dat aangeeft hoe waarschijnlijk het is dat de gebeurtenis plaatsvindt. Een kans van 0 betekent dat de gebeurtenis nooit zal gebeuren, terwijl een kans van 1 betekent dat het altijd gebeurt. Bijvoorbeeld, bij een eerlijke dobbelsteen is de kans om een drie te gooien 1 op 6, oftewel $P(⚂) = \frac{1}{6}$, omdat er zes mogelijke uitkomsten zijn die allemaal even waarschijnlijk zijn om op te treden, en slechts één daarvan is een drie. De kans om een even getal te gooien met een dobbelsteen is $P(\text{even}) = \frac{3}{6} = \frac{1}{2}$ omdat drie van de zes mogelijke uitkomsten even zijn.

Kansrekening omvat ook verschillende regels en concepten om complexere situaties te begrijpen. Zo zijn er regels die ons vertellen hoe we de kans berekenen dat meerdere gebeurtenissen plaatsvinden.

Bijvoorbeeld, als je de kans wil weten dat je bij het gooien van een dobbelsteen een één of een twéé gooit, tel je de afzonderlijke kansen bij elkaar op:

$$
P(⚀ \cup ⚁) = P(⚀) + P(⚁)
$$

Het $\cup$-symbool staat voor "of" (de vereniging van twee verzamelingen). Deze regel gaat alleen op als het kansen zijn op gebeurtenissen die nooit of te nimmer allebei samen kunnen optreden: als je een één gooit, gooi je nooit tegelijkertijd een twéé.

Daarentegen, als je de kans wil weten dat je bij het gooien van een dobbelsteen eerst een één en daarna met een nieuwe worp een twéé gooit, vermenigvuldig je de afzonderlijke kansen:

$$
P(⚀ \cap ⚁) = P(⚀) \cdot P(⚁)
$$

Het $\cap$-symbool staat voor "en" (de doorsnede van twee verzamelingen). Deze regel gaat alleen op als het kansen zijn die onafhankelijk van elkaar optreden en niet van elkaar afhangen: het resultaat van een eerste worp zegt helemaal niets over dat van een tweede worp.
 
Een ander belangrijk concept is de voorwaardelijke kans, die ons helpt te begrijpen hoe de kans op een gebeurtenis verandert als we al iets weten over een andere gebeurtenis. Stel je voor dat je een kaart trekt uit een standaard kaartspel, en je weet dat de kaart zwart is. De kans dat deze kaart een schoppen twee is, is anders dan de oorspronkelijke kans voordat je wist dat de kaart zwart was.

$$
\begin{align}
P(♠2) = \frac{1}{52}
\\
P(♠2 | \text{zwart}) = \frac{1}{26}
\end{align}
$$

Het |-symbool staat voor "gegeven dat". Dit wordt ook wel een *voorwaardelijke* kans genoemd.

Voorwaardelijke kansen zijn nuttig als je gebeurtenissen wil combineren die wél met elkaar samenhangen. Stel bijvoorbeeld dat ik de kans wil weten dat ik in één worp van een dobbelsteen een aantal ogen gooi dat zowel even is als groter dan drie. Je kan voor dit eenvoudige voorbeeld nagaan dat er twee uitkomsten even én groter dan drie zijn (⚃ en ⚅), dus die kans is

$$
P(\text{ even } \cap \text{ >3 }) = \frac{2}{6} = \frac{1}{3}
$$

Echter, de kans dat een worp even is is $P(\text{ even }) = \frac{3}{6} = \frac{1}{2}$ en de kans dat een uitkomst groter dan drie is is $P(\text{ >3 }) = \frac{3}{6} = \frac{1}{2}$, dus in dit geval is dit niet gelijk aan het product van de losse kansen! Dit komt omdat de kansen niet onafhankelijk zijn: als je even gooit is de kans groter dat je groter dan drie gooit dan dat je minder gooit.

Je kan kansen echter wel combineren met de productregel als je voorwaardelijke kansen gebruikt: $P(A \cap B) = P(A) \cdot P(B|A)$, of ook $P(A \cap B) = P(B) \cdot P(A|B)$. Zo is hier $P(\text{ >3 } | \text{ even }) = \frac{2}{3}$. Ga zelf na hoe groot $P(\text{ even } | \text{ >3 })$ is.

Met deze basisprincipes kunnen we complexere modellen doorrekenen, zoals Hidden Markov Modellen. Een Hidden Markov Model (HMM) maakt gebruik van kansrekening om sequenties van observaties te analyseren en te voorspellen. Het model gaat uit van een reeks verborgen toestanden die niet direct zichtbaar zijn, en een reeks waargenomen uitkomsten die afhankelijk zijn van deze toestanden. HMM's gebruiken transitiewaarschijnlijkheden om te berekenen hoe groot de kans is dat het systeem van de ene verborgen toestand naar de andere overgaat, en emissiekansen om te bepalen hoe waarschijnlijk een bepaalde waarneming is gegeven een specifieke verborgen toestand. Door deze kansen te combineren, kan een HMM bepalen hoe waarschijnlijk een gegeven reeks observaties is, of de meest waarschijnlijke reeks verborgen toestanden achter de reeks observaties bepalen.

[Khan Academy](https://www.khanacademy.org/math/math2/xe2ae2386aa2e13d6:prob) presenteert een multimediale les over basisbegrippen in de kansrekening (**Unit 13: Probability**). Doorloop zelfstandig deze les, bekijk voor zover je dat nodig acht de video's, vul de oefeningen in, en doe de quizes. Je dient de afsluitende [Unit test](https://www.khanacademy.org/math/math2/xe2ae2386aa2e13d6:prob/xe2ae2386aa2e13d6:expected-value/test/xe2ae2386aa2e13d6:prob-unit-test?referrer=upsell) met succes af te ronden.

In [4]:
# UITWERKING

<a id="3" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### Experimentele waarschijnlijkheid

De vorige les heb je een reeks knikkers getrokken aan drie verschillende tafels. De gegevens omtrent overgangswaarschijnlijkheden en emissiekansen kun je aflezen uit de bekende gegevens omtrent de opzet van het experiment:

| Tafel: |  ❶  |  ❷  |  ❸  |
| -----: | :-: | :-: | :-: |
| **Grabbelton:** | 6x blauw | 2x blauw | 1x blauw |
|                 | 3x geel  | 6x geel  | 0x geel  | 
|                 | 1x groen | 2x groen | 6x groen |
|                 | 2x rood  | 2x rood  | 5x rood  |
| **Dobbelsteen:** | ⚀→① | ⚀→① | ⚀→① |
|                  | ⚁→② | ⚁→② | ⚁→① |
|                  | ⚂→② | ⚂→② | ⚂→① |
|                  | ⚃→② | ⚃→③ | ⚃→① |
|                  | ⚄→③ | ⚄→③ | ⚄→② |
|                  | ⚅→③ | ⚅→③ | ⚅→③ |

Laten we eens proberen te berekenen wat de kans is op de volgende reeks van vijf uitkomsten:

| **Beurt:** | 1     | 2     | 3     | 4     | 5     |
| ---------: | :---: | :---: | :---: | :---: | :---: |
| **Tafel:** | ❷     | ❸     | ❶     | ❸     | ❷     |
| **Kleur:** | geel  | groen | blauw | rood  | groen |

Merk op dat een Hidden Markov Model aanneemt dat we de toestanden (de tafels) niet kunnen waarnemen. In dit geval kennen we de tafels echter wel. We berekenen daarom voorlopig de kans op al deze kleuren én al deze tafels samen.

##### Tafel ❷ in beurt 1

Je bent aan een willekeurige tafel begonnen, dus de kans op elke tafel in beurt 1 was aanvankelijk gelijk. Dat betekent dat de kans dat je in beurt 1 aan tafel ❷ zou belanden gelijk is aan

$$
P(❷_1) = \frac{1}{3}
$$

Die kans hangt verder nergens van af. Hierboven wordt een onderschrift gebruikt om het nummer van de beurt aan te geven.

##### Kleur geel in beurt 1

Gegeven nu dat je in beurt 1 aan tafel ❷ zat, kon je 6 verschillende gele knikkers trekken uit een totaal van 12 knikkers. Daarom is

$$
P(\text{geel}_1|❷_1) = \frac{6}{12}
$$

Deze kans hangt alléén af van het feit dat je in deze beurt aan tafel ❷ zat, en de kans dat dat aan de hand is hadden we hiervoor al berekend.

##### Tafel ❸ in beurt 2

Gegeven dat je in beurt 1 aan tafel ❷ zat, waren er drie uitkomsten van de dobbelsteen die je voor beurt 2 naar tafel ❸ zouden sturen. Daarom is

$$
P(❸_2|❷_1) = \frac{3}{6}
$$

Ook deze kans hangt alléén af van het feit dat je hiervoor aan tafel ❷ zat, en dat hebben we hierboven al doorgerekend. Het hangt bijvoorbeeld niet af van het feit dat je hiervoor een gele knikker getrokken had.

##### Kleur groen in beurt 2

Gegeven dat je aan tafel ❸ zat, kon je wederom 6 groene knikkers trekken uit een totaal van 12 knikkers. Daarom is

$$
P(\text{groen}_2|❸_2) = \frac{6}{12}
$$

Deze kans hangt wederom alléén af van het feit dat je in deze beurt aan tafel ❸ zat, en die kans werd in de stap hiervoor berekend. De tafel of de kleur van de vorige beurt heeft geen rechtstreeks effect op deze uitkomst.

##### Enzovoorts

Voor de beurten daarna kunnen we op soortgelijke manier de kansen berekenen. Dankzij de aannamen van een Hidden Markov Model hangt de kans op een tafel in een beurt alléén af van de tafel in de vorige beurt, en de kleur van een knikker in een beurt alléén van de tafel in diezelfde beurt. We krijgen dan

$$
\begin{align}
P(❶_3|❸_2) &= \frac{4}{6}
\\
P(\text{blauw}_3|❶_3) &= \frac{6}{12}
\\
P(❸_4|❶_3) &= \frac{2}{6}
\\
P(\text{rood}_4|❸_4) &= \frac{5}{12}
\\
P(❷_5|❸_4) &= \frac{1}{6}
\\
P(\text{groen}_5|❷_5) &= \frac{2}{12}
\end{align}
$$

##### Tezamen

We kunnen nu de kansen hierboven gebruiken om te berekenen hoe groot de kans op de verkregen tafels en kleuren uit alle vijf de beurten samen is geweest. We zijn dan eigenlijk op zoek naar

$$
p = P(❷_1 \cap \text{geel}_1 \cap ❸_2 \cap \text{groen}_2 \cap ❶_3 \cap \text{blauw}_3 \cap ❸_4 \cap \text{rood}_4 \cap ❷_5 \cap \text{groen}_5 )
$$

Deze kansen zijn niet allemaal onafhankelijk van elkaar, maar we kunnen ze wel zo schrijven dat elke kans alleen afhangt van de gebeurtenissen eerder in de reeks. Daarom mogen we dit schrijven als een product

$$
p = P(❷_1) \cdot P(\text{geel}_1|❷_1) \cdot P(❸_2|❷_1) \cdot P(\text{groen}_2|❸_2) \cdot P(❶_3|❸_2) \cdot P(\text{blauw}_3|❶_3) \cdot P(❸_4|❶_3) \cdot P(\text{rood}_4|❸_4) \cdot P(❷_5|❸_4) \cdot P(\text{groen}_5|❷_5)
$$

En dit kunnen we nu uitrekenen als

$$
p = \frac{1}{3} \cdot \frac{6}{12} \cdot \frac{3}{6} \cdot \frac{6}{12} \cdot \frac{4}{6} \cdot \frac{6}{12} \cdot \frac{2}{6} \cdot \frac{5}{12} \cdot \frac{1}{6} \cdot \frac{2}{12} = \frac{51840}{967458816} = \frac{5}{93312} \approx 0.000054
$$

Deze kans is erg klein, wat op zich niet verbazingwekkend is omdat de kans op elke specifieke reeks waarnemingen van tafels en kleuren niet erg groot kan zijn. Er zijn immers enorm veel verschillende reeksen tafels en kleuren die waargenomen hadden kunnen worden (maar niet elke reeks is even aannemelijk).

Hoe langer de reeks waarnemingen, hoe kleiner deze kans wordt. Op den duur kan de kans zo ver afnemen dat floating-point waarden in een computer niet meer in staat zijn om deze minieme kansen goed te representeren: ze worden dan afgerond naar nul. Om dat te voorkomen wordt soms gewerkt met de logaritme van de kans: de *log-waarschijnlijkheid*. De rekenregels van logaritmen zeggen dat de logaritme van een product van getallen kan worden geschreven als de som van logaritmen van die getallen. De formule hierboven wordt dan

$$
\ln \left( p \right) = \ln \left( P(\text{geel}_1|❷_1) \right) + \ln \left( P(\text{geel}_1|❷_1) \right) + \ldots + \ln \left( P(\text{groen}_5|❷_5) \right)
$$

In plaats van waarschijnlijkheden te vermenigvuldigen kunnen we dus de bijbehorende log-waarschijnlijkheden simpelweg optellen. De uitkomsten worden hierbij niet zo extreem. Dit komt uiteindelijk neer op

$$
\ln \left( p \right) \approx -9.834
$$

Bereken nu voor de eerste vijf beurten uit jouw eigen experimentele reeks de waarschijnlijkheid $p$ en de log-waarschijnlijkheid $\ln \left( p \right)$. Wissel je reeks uit met je groepsgenoot en controleer elkaars resultaten. Hebben jullie allebei een (ongeveer) even waarschijnlijke reeks waarnemingen verkregen?

In [5]:
# UITWERKING

<a id="4" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### Je eigen `HiddenMarkovModel` class

De vorige les ben je begonnen met het aanmaken van een eigen klasse `HiddenMarkovModel`. Voeg nu een methode `score()` toe die twee argumenten ontvangt: een iterable met emissies (hier bijvoorbeeld de reeks kleuren; `X`, als we `hmmlearn` naamgeving volgen) en een iterable met toestanden (hier bijvoorbeeld de reeks tafels; `state_sequence`, volgens `hmmlearn`). Iterables zijn datastructuren waar je met een for-loop doorheen kan itereren; bijvoorbeeld python lijsten, tuples, of numpy-arrays. Je mag aannemen dat de emissies en toestanden gecodeerd zijn als python integers (bijvoorbeeld de kleuren blauw/geel/groen/rood als `0, 1, 2, 3` en de tafels ❶/❷/❸ als `0, 1, 2`). De methode dient op grond van de bekende begintoestandverdeling, emissiekansen, en overgangswaarschijnlijkheden te berekenen hoe groot de log-waarschijnlijkheid is op die reeks waarnemingen, en dat als retourwaarde teruggeven aan de gebruiker.

Gebruik je eigen module om de berekening uit de voorgaande oefening te controleren. Komt er hetzelfde antwoord uit? Bereken ook de log-waarschijnlijkheid voor je hele reeks eigen waarnemingen, en voor de hele reeks waarnemingen van de hele klas; deze berekeningen zijn wat omslachtig om met de hand te doen, maar je klasse zou het met gemak aan moeten kunnen.

De `hmmlearn` module definieert een soortgelijke `score()` methode die weliswaar de emissies `X` ontvangt, maar niet de toestanden `state_sequence`. Toestanden worden in een Hidden Markov Model eigenlijk niet waarneembaar verondersteld. We kunnen echter met onze eigen module hetzelfde resultaat bereiken als met `hmmlearn` door de kansen te sommeren over *alle mogelijke* reeksen toestanden. Als je Hidden Markov Model geïnitialiseerd is in de variabele `model` krijg je iets als het volgende:

```python
from itertools import product
from math import exp, log

prob_sum = 0.0
for state_sequence in product(range(3), repeat=5):   # 5 herhalingen van 3 mogelijke toestanden
    prob_sum += exp(model.score(X, state_sequence))
log_prob = log(prob_sum)
```

Het resultaat hiervan zou dan hetzelfde moeten opleveren (wellicht op afrondfoutjes na) als een aanroep zoals `CategoricalHMM(...).score(X)` uit de `hmmlearn` module. Controleer dat de uitkomsten van je eigen module hiermee overeenkomen. Lukt het om de scores voor je eigen eerste vijf waarnemingen te bepalen, je volledige eigen reeks waarnemingen, of de reeks waarnemingen van de hele klas? Merk op dat de bovenstaande for-loop al snel enorm traag kan worden zodra het aantal waarnemingen toeneemt; we gaan dit gedrag in een latere les verbeteren.

In [6]:
# UITWERKING

States    : [1 2 0 2 1]
Emissions : [1 2 0 3 2]
Eigen module:   ln(p) = -9.834   (p = 5.4e-05)
Dit komt overeen met de handmatige berekening!

Som over ALLE mogelijke States:
Eigen module:   ln(p) = -6.578   (p = 1.4e-03)
hmmlearn:       ln(p) = -6.578   (p = 1.4e-03)
Het resultaat van de eigen module komt overeen met dat van hmmlearn!


<a id="5" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### CpG-eilandjes

In het vorige deel heb je aan de hand van realistische transitiewaarschijnlijkheden willekeurige sequenties gegenereerd van 300 nucleotiden lang die overeenkomen met een lokatie in of buiten een CpG-eilandje. Neem die sequenties hier over, of genereer ze opnieuw.

Instantieer ook twee Hidden Markov Modellen die elk vier toestanden hebben: A, C, T, en G. De overgangswaarschijnlijkheden kun je nu kiezen voor het ene model zoals ze zijn binnen een CpG-eilandje en voor het andere model zoals ze zijn erbuiten (zie de tabellen in deel I.3 *CpG-eiland en non-CpG-eiland sequenties*). In dit geval komen de toestanden exact overeen met de waarnemingen, dus de emissiekansen kunnen we als volgt instellen.

|  `+`  |   A   |   C   |   G   |   T   |
| :---: | :---: | :---: | :---: | :---: |
| **A** |  1.0  |  0.0  |  0.0  |  0.0  |
| **C** |  0.0  |  1.0  |  0.0  |  0.0  |
| **T** |  0.0  |  0.0  |  1.0  |  0.0  |
| **G** |  0.0  |  0.0  |  0.0  |  1.0  |

De toestanden staan in de rijen, en de emissies in de kolommen. Deze tabel geeft aan dat toestand **A** altijd een **A** als emissie geeft, etc. Eigenlijk is dit niet zozeer een Hidden Markov Model, maar een [Markov Chain](https://en.wikipedia.org/wiki/Markov_chain). Immers, de toestanden zijn niet onbekend ("hidden"), maar komen precies overeen met de waargenomen emissies. Echter, een Hidden Markov model is algemener dan een Markov Chain en kan ook hiermee omgaan.

Je hebt nu dus een Hidden Markov Model dat de kans kan bepalen op een bepaalde sequentie als die binnen een CpG-eilandje voorkomt, en een ander model dat de kans bepaalt op de sequentie buiten een CpG-eilandje.

Pas nu beide modellen toe op beide gegenereerde random sequenties. Je krijgt dus vier uitkomsten. Geeft het model dat past binnen een CpG-eilandje ook de hoogste kans aan de sequentie die past binnen een CpG-eilandje? En omgekeerd, geeft het model dat past buiten een CpG-eilandje ook de hoogste kans aan de sequentie die past buiten een CpG-eilandje?

Als uitdaging, kun je de eigenschappen van beide Hidden Markov Modellen combineren in één model? Dat wil zeggen, kun je een model maken dat zowel sequenties binnen en buiten een CpG-eilandje kan verwerken en alle log-waarschijnlijkheden die je hierboven berekende kan evalueren? Waarom krijg je niet per se exact hetzelfde antwoord? (Dit model hoeft niet om te kunnen gaan met sequenties waarvan sommige delen binnen en andere delen buiten CpG-eilandjes liggen.) Hint: maak een model met acht verschillende toestanden en vier verschillende emissies.

Tenslotte, gegeven is de volgende sequentie:

```
TCCCCGCAGGCCATAGCCCGGGACGTCCGACAGCCGGCTGGTGCTGGGGGTAGGCATAATCGCGAGAGCCACCGTCGTCTGTCTGCCTGCTGAGCCTTAG
```

Komt deze uit een CpG-eilandje, of van ergens daarbuiten? Kun je iets zinnigs zeggen over hoe zeker je bent van dit antwoord?

In [7]:
# UITWERKING

CGI+ Sequence:
GGACACGCTATGGGAGGGGGGGCCCCAACCCCCGCCGCCCGCTCCTGGACTGCGCCTCCCCCAGGGGCCCGAGAAAAGGGTGCCGGGCGCAGCATGCGCTTGGAAGCCGCGCGGTGTTGATGCCCCCGGACGCCCACAGGGTCGCGACGCGCTTCTACCTCTCCAGACTCTTCTGGACACTCCTGGAAGGGACACCAGGTGTGCCGAGCCTCCCCCTGCGGCCACGTCGCTGGGGCGCCCTGGCCGCGGCAGCCCCCCTGGCAGTGTTCGCCCTAGGAATGTCCCAAGCGTACCTCGGCC

CGI- sequence:
AAGTGGGCCTTCCCTTTGAAAGCTGCGTAGACACCCCATGGATGTAGGAGTTAAAATTCTGCACAACCTAAATCTAATGCGTGGAAGAAGTTCCTGGGATTTTGGGGAAAAGGCCAGGAATGCTATTTTGCACATCTGCCTAGGACCTGGCTTTGAACCAAGGGATAAATTCCTTTGATAGTGAATTAGTGGTAGTCAGAAAGATCAACCAAGCCTCATGCCAGCACATGGGCCCAGCCGTCTGACTTAATCATCTCAATGGAAACAAAGCTCACAGAGAGTGAACCACATGGAGCCTAA

CGI+ seq volgens CGI+ model:	ln(p) = -383.374
CGI- seq volgens CGI+ model:	ln(p) = -443.261
CGI+ seq volgens CGI- model:	ln(p) = -426.543
CGI- seq volgens CGI- model:	ln(p) = -400.798

Volgens gecombineerd model:
CGI+ seq in CGI+ gebied:	ln(p) = -384.067
CGI- seq in CGI+ gebied:	ln(p) = -443.955
CGI+ seq in CGI- gebied:	ln(p) = -427.236
CGI- seq in CGI- gebied:	ln

***

&copy; 2024 - Dave R.M. Langers <d.r.m.langers@pl.hanze.nl>